# Wrangling and Analyzing WeRateDogs Twitter Account (DAND project)
Abdullah Almuzaini

## Introduction 


##  Data Gathering

> Gathering the Pieces of datasets required for the project. In this case, three datasets will be fathered and dowlnloaded into the notebook
> The three datasets are:
>    - WeRateDogs twitter account archive, which is hosted on Udacity server on this link. 

#### Importing the necessary libraries

In [1]:
import requests
import os
import pandas as pd
import tweepy
import json
from tqdm import tqdm
pd.set_option('display.max_rows', None, 'display.max_columns',None)
import numpy as np
import re
import ast


### Below cell contains two functions:
> - new_folder() to create new folder
> - get_files() to download files from the internet programmticly 




In [2]:

def new_folder(folder_name): 
    """
    this funcation creates a new directory in the currnet running directory when it's called
    """
    if not os.path.exists(folder_name):
        os.makedirs(folder_name)


def get_files(file_name, url):
    """
    The following function download any file from the internet using requests library and save it in the 'dataset' directory. 
    note about the file_name: 
    the file_name var has to include the file format


    """ 
    response = requests.get(url)
    with open(os.path.join('dataset', url.split('/')[-1]), mode='wb') as file:
        file.write(response.content)


In [3]:
# Create a new folder if it does not exist 
new_folder('dataset')

In [4]:
URL = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/59a4e958_twitter-archive-enhanced/twitter-archive-enhanced.csv'

data = get_files(file_name = "twitter-archive-enhanced.csv", url= URL  )
df = pd.read_csv('dataset'+ '/'+'twitter-archive-enhanced.csv', parse_dates=True)
df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None


In [5]:
# The URL to `image-predictions.tsv` dataset

URL = 'https://d17h27t6h515a5.cloudfront.net/topher/2017/August/599fd2ad_image-predictions/image-predictions.tsv'

# Download the dataset 

data = get_files(file_name = "image-predictions.tsv", url= URL)

# Read the dataset in pandas DataFrame
img_pred = pd.read_csv('dataset/image-predictions.tsv', sep='\t')
img_pred.head()


,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
0,666020888022790149,https://pbs.twimg.com/media/CT4udn0WwAA0aMy.jpg,1,Welsh_springer_spaniel,0.465074,True,collie,0.156665,True,Shetland_sheepdog,0.061428,True
1,666029285002620928,https://pbs.twimg.com/media/CT42GRgUYAA5iDo.jpg,1,redbone,0.506826,True,miniature_pinscher,0.074192,True,Rhodesian_ridgeback,0.072010,True
2,666033412701032449,https://pbs.twimg.com/media/CT4521TWwAEvMyu.jpg,1,German_shepherd,0.596461,True,malinois,0.138584,True,bloodhound,0.116197,True
3,666044226329800704,https://pbs.twimg.com/media/CT5Dr8HUEAA-lEu.jpg,1,Rhodesian_ridgeback,0.408143,True,redbone,0.360687,True,miniature_pinscher,0.222752,True
4,666049248165822465,https://pbs.twimg.com/media/CT5IQmsXIAAKY4A.jpg,1,miniature_pinscher,0.560311,True,Rottweiler,0.243682,True,Doberman,0.154629,True


In [6]:
img_pred.shape

(2075, 12)

### Setting up tweetpy 

In [8]:
# Twitter Credentials

ACCESS_TOKEN = "1235133671851601920-sN4bWU6RDTydw57nqVDxwQfAvzam2J"  
ACCESS_TOKEN_SECRET = "zySb7dmbCFRKOIE1RyK07NhUC16ZGWgd9lvFJoQmtFbuL"  
CONSUMER_KEY = "JFo7yOJ5q4OssR9LNuekFJmZh" 
CONSUMER_SECRET = "5npceXl51epMWcIeqc4b20ss8FqOeCEX2NkpY1nniR7Zf69uBM"

auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)

api = tweepy.API(auth,wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

In [9]:
# Test the api
tweet = api.get_status(677187300187611136)
print(tweet.user.screen_name)

dog_rates


> #### The following block of code will collect three pieces of data about each tweet
> and these pieces of data are:
>    - tweet_id (will be brought from 'twitter-archive-enhanced.csv'
>    - retweet_count (will be collected using tweetpy.api)
>    - favorite_count (will be collected using tweetpy.api)

> #### These pieces of data will be added to a dict var in order to save them in json format


In [58]:
data = {}
data['tweet_info']=  []
for i in tqdm(df['tweet_id']):
    try:
        tweet = api.get_status(i)
        data['tweet_info'].append({
            'tweet_id' : i ,
            'retweet_count': tweet.retweet_count ,
            'likes_count' : tweet.favorite_count
        })
    except:
        print("The tweet associated to the tweet id", i," was not found")


  1%|          | 21/2356 [00:04<09:14,  4.21it/s]

The tweet associated to the tweet id 888202515573088257  was not found


  4%|▍         | 97/2356 [00:18<07:07,  5.28it/s]

The tweet associated to the tweet id 873697596434513921  was not found


  4%|▍         | 103/2356 [00:19<06:31,  5.75it/s]

...

 40%|███▉      | 934/2356 [15:11<04:07,  5.74it/s]    

The tweet associated to the tweet id 754011816964026368  was not found


 73%|███████▎  | 1727/2356 [17:37<01:41,  6.20it/s]

The tweet associated to the tweet id 680055455951884288  was not found


100%|██████████| 2356/2356 [31:53<00:00,  1.23it/s]    


In [6]:
# Create a text file in the current directory and save each tweet's retweet count and favorite ("like") count
# in text file as json format

with open('tweet_json1.txt', 'w') as outfile:
    json.dump(data, outfile)

> ### Below, I will load "tweet_json.txt" into the notebook and print it out to see how it looks

In [10]:
# Read 'tweet.txt' line by line
with open('tweet_json.txt') as json_file:
    data = json.load(json_file)
    for p in data['tweet_info']:
        print('Tweet id: ' , p['tweet_id'])
        print('Retweet count: ' , p['retweet_count'])
        print('Likes count: ' , p['likes_count'])
        print('')

Tweet id:  892420643555336193
Retweet count:  7189
Likes count:  34531

Tweet id:  892177421306343426
Retweet count:  5396
Likes count:  29940

Tweet id:  891815181378084864
Retweet count:  3562
Likes count:  22530

Tweet id:  891689557279858688
Retweet count:  7413
Likes count:  37769

Tweet id:  891327558926688256
Retweet count:  7947
Likes count:  36071

Tweet id:  891087950875897856
Retweet count:  2672
Likes count:  18197

Tweet id:  890971913173991426
Retweet count:  1715
Likes count:  10603

Tweet id:  890729181411237888
Retweet count:  16176
Likes count:  58118

Tweet id:  890609185150312448
Retweet count:  3712
Likes count:  25055

Tweet id:  890240255349198849
Retweet count:  6247
Likes count:  28574

Tweet id:  890006608113172480
Retweet count:  6296
Likes count:  27595

Tweet id:  889880896479866881
Retweet count:  4274
Likes count:  25085

Tweet id:  889665388333682689
Retweet count:  8560
Likes count:  42978

Tweet id:  889638837579907072
Retweet count:  3818
Likes count:

Retweet count:  1257
Likes count:  10494

Tweet id:  838476387338051585
Retweet count:  4463
Likes count:  21486

Tweet id:  838201503651401729
Retweet count:  9442
Likes count:  0

Tweet id:  838150277551247360
Retweet count:  313
Likes count:  1596

Tweet id:  838085839343206401
Retweet count:  1
Likes count:  134

Tweet id:  838083903487373313
Retweet count:  2920
Likes count:  16870

Tweet id:  837820167694528512
Retweet count:  7353
Likes count:  32564

Tweet id:  837482249356513284
Retweet count:  417
Likes count:  3693

Tweet id:  837471256429613056
Retweet count:  2119
Likes count:  12259

Tweet id:  837110210464448512
Retweet count:  2228
Likes count:  15411

Tweet id:  836989968035819520
Retweet count:  2168
Likes count:  12238

Tweet id:  836753516572119041
Retweet count:  4363
Likes count:  18538

Tweet id:  836677758902222849
Retweet count:  2083
Likes count:  12112

Tweet id:  836648853927522308
Retweet count:  519
Likes count:  0

Tweet id:  836397794269200385
Retweet co

Likes count:  7311

Tweet id:  812709060537683968
Retweet count:  1383
Likes count:  6440

Tweet id:  812503143955202048
Retweet count:  1178
Likes count:  5890

Tweet id:  812466873996607488
Retweet count:  1838
Likes count:  7744

Tweet id:  812372279581671427
Retweet count:  3516
Likes count:  13232

Tweet id:  811985624773361665
Retweet count:  1345
Likes count:  7069

Tweet id:  811744202451197953
Retweet count:  1528
Likes count:  7326

Tweet id:  811647686436880384
Retweet count:  707
Likes count:  5445

Tweet id:  811627233043480576
Retweet count:  2939
Likes count:  12353

Tweet id:  811386762094317568
Retweet count:  6126
Likes count:  20317

Tweet id:  810984652412424192
Retweet count:  1367
Likes count:  5163

Tweet id:  810896069567610880
Retweet count:  1686
Likes count:  8739

Tweet id:  810657578271330305
Retweet count:  2526
Likes count:  10650

Tweet id:  810284430598270976
Retweet count:  10780
Likes count:  34136

Tweet id:  810254108431155201
Retweet count:  3245
L


Tweet id:  783085703974514689
Retweet count:  2088
Likes count:  7897

Tweet id:  782969140009107456
Retweet count:  7060
Likes count:  23376

Tweet id:  782747134529531904
Retweet count:  1353
Likes count:  7236

Tweet id:  782722598790725632
Retweet count:  5146
Likes count:  16686

Tweet id:  782598640137187329
Retweet count:  1803
Likes count:  7558

Tweet id:  782305867769217024
Retweet count:  5293
Likes count:  16091

Tweet id:  782021823840026624
Retweet count:  5920
Likes count:  0

Tweet id:  781955203444699136
Retweet count:  3209
Likes count:  10804

Tweet id:  781661882474196992
Retweet count:  2593
Likes count:  10104

Tweet id:  781655249211752448
Retweet count:  1084
Likes count:  3865

Tweet id:  781524693396357120
Retweet count:  5297
Likes count:  20061

Tweet id:  781308096455073793
Retweet count:  2447
Likes count:  6906

Tweet id:  781251288990355457
Retweet count:  1984
Likes count:  8112

Tweet id:  781163403222056960
Retweet count:  2635
Likes count:  9490

Tw

Retweet count:  3722
Likes count:  9085

Tweet id:  746906459439529985
Retweet count:  275
Likes count:  2772

Tweet id:  746872823977771008
Retweet count:  2009
Likes count:  5698

Tweet id:  746818907684614144
Retweet count:  1645
Likes count:  5058

Tweet id:  746790600704425984
Retweet count:  1509
Likes count:  4624

Tweet id:  746757706116112384
Retweet count:  3678
Likes count:  9082

Tweet id:  746726898085036033
Retweet count:  1685
Likes count:  5765

Tweet id:  746542875601690625
Retweet count:  1784
Likes count:  4796

Tweet id:  746521445350707200
Retweet count:  926
Likes count:  0

Tweet id:  746507379341139972
Retweet count:  1023
Likes count:  4439

Tweet id:  746369468511756288
Retweet count:  1533
Likes count:  5911

Tweet id:  746131877086527488
Retweet count:  2056
Likes count:  6542

Tweet id:  746056683365994496
Retweet count:  763
Likes count:  3397

Tweet id:  745789745784041472
Retweet count:  993
Likes count:  3792

Tweet id:  745712589599014916
Retweet count

Tweet id:  707629649552134146
Retweet count:  814
Likes count:  2428

Tweet id:  707610948723478529
Retweet count:  5920
Likes count:  16055

Tweet id:  707420581654872064
Retweet count:  655
Likes count:  2129

Tweet id:  707411934438625280
Retweet count:  562
Likes count:  2182

Tweet id:  707387676719185920
Retweet count:  1188
Likes count:  3307

Tweet id:  707377100785885184
Retweet count:  1010
Likes count:  3088

Tweet id:  707315916783140866
Retweet count:  595
Likes count:  2341

Tweet id:  707297311098011648
Retweet count:  734
Likes count:  2644

Tweet id:  707059547140169728
Retweet count:  634
Likes count:  2465

Tweet id:  707038192327901184
Retweet count:  736
Likes count:  2068

Tweet id:  707021089608753152
Retweet count:  1272
Likes count:  3869

Tweet id:  707014260413456384
Retweet count:  558
Likes count:  2187

Tweet id:  706904523814649856
Retweet count:  7336
Likes count:  13784

Tweet id:  706901761596989440
Retweet count:  543
Likes count:  1935

Tweet id:  70

Retweet count:  1011
Likes count:  2417

Tweet id:  689154315265683456
Retweet count:  925
Likes count:  2925

Tweet id:  689143371370250240
Retweet count:  468
Likes count:  1914

Tweet id:  688916208532455424
Retweet count:  804
Likes count:  2563

Tweet id:  688908934925697024
Retweet count:  719
Likes count:  1990

Tweet id:  688898160958271489
Retweet count:  731
Likes count:  1983

Tweet id:  688894073864884227
Retweet count:  641
Likes count:  2126

Tweet id:  688828561667567616
Retweet count:  342
Likes count:  1306

Tweet id:  688804835492233216
Retweet count:  176
Likes count:  913

Tweet id:  688789766343622656
Retweet count:  623
Likes count:  2102

Tweet id:  688547210804498433
Retweet count:  657
Likes count:  2455

Tweet id:  688519176466644993
Retweet count:  685
Likes count:  2203

Tweet id:  688385280030670848
Retweet count:  4189
Likes count:  8773

Tweet id:  688211956440801280
Retweet count:  783
Likes count:  2126

Tweet id:  688179443353796608
Retweet count:  547

Tweet id:  676949632774234114
Retweet count:  367
Likes count:  1222

Tweet id:  676948236477857792
Retweet count:  795
Likes count:  2007

Tweet id:  676946864479084545
Retweet count:  337
Likes count:  1612

Tweet id:  676942428000112642
Retweet count:  686
Likes count:  1928

Tweet id:  676936541936185344
Retweet count:  4565
Likes count:  11899

Tweet id:  676916996760600576
Retweet count:  1671
Likes count:  2762

Tweet id:  676897532954456065
Retweet count:  667
Likes count:  2099

Tweet id:  676864501615042560
Retweet count:  650
Likes count:  1951

Tweet id:  676821958043033607
Retweet count:  14220
Likes count:  21272

Tweet id:  676819651066732545
Retweet count:  611
Likes count:  1696

Tweet id:  676811746707918848
Retweet count:  396
Likes count:  1341

Tweet id:  676776431406465024
Retweet count:  1855
Likes count:  4651

Tweet id:  676617503762681856
Retweet count:  880
Likes count:  2685

Tweet id:  676613908052996102
Retweet count:  184
Likes count:  1012

Tweet id:  67

Likes count:  672

Tweet id:  670792680469889025
Retweet count:  244
Likes count:  753

Tweet id:  670789397210615808
Retweet count:  215
Likes count:  596

Tweet id:  670786190031921152
Retweet count:  180
Likes count:  546

Tweet id:  670783437142401025
Retweet count:  344
Likes count:  754

Tweet id:  670782429121134593
Retweet count:  690
Likes count:  1399

Tweet id:  670780561024270336
Retweet count:  260
Likes count:  722

Tweet id:  670778058496974848
Retweet count:  65
Likes count:  304

Tweet id:  670764103623966721
Retweet count:  384
Likes count:  985

Tweet id:  670755717859713024
Retweet count:  95
Likes count:  399

Tweet id:  670733412878163972
Retweet count:  463
Likes count:  876

Tweet id:  670727704916926465
Retweet count:  323
Likes count:  757

Tweet id:  670717338665226240
Retweet count:  432
Likes count:  1086

Tweet id:  670704688707301377
Retweet count:  333
Likes count:  705

Tweet id:  670691627984359425
Retweet count:  218
Likes count:  533

Tweet id:  6706

> #### Convert 'tweet_json.txt' to a pandas dataframe 

In [11]:
# Read 'tweet.txt' line by line into a pandas dataframe with tweet_id, retweet_count, and likes_count
tweet_json_df = pd.json_normalize(data['tweet_info'])


In [12]:
tweet_json_df.head()

,tweet_id,retweet_count,likes_count
0,892420643555336193,7189,34531
1,892177421306343426,5396,29940
2,891815181378084864,3562,22530
3,891689557279858688,7413,37769
4,891327558926688256,7947,36071



## Assessing Data


### Assessing WeRateDogs twitter account archive table



In [13]:
df.sample(20)

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo
1325,706166467411222528,NaN,NaN,2016-03-05 17:16:20 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Dexter. He's a shy pup. Doesn't bark m...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/706166467...,7,10,Dexter,None,None,None,None
821,770293558247038976,NaN,NaN,2016-08-29 16:14:30 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Jackson. There's nothing abnormal abou...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/770293558...,10,10,Jackson,None,None,None,None
780,775733305207554048,NaN,NaN,2016-09-13 16:30:07 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Anakin. He strives to reach his full d...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/775733305...,11,10,Anakin,doggo,None,None,None
1549,689255633275777024,NaN,NaN,2016-01-19 01:18:43 +0000,"<a href=""http://vine.co"" rel=""nofollow"">Vine -...",This is Ferg. He swallowed a chainsaw. 1 like ...,NaN,NaN,NaN,https://vine.co/v/iOL792n5hz2,10,10,Ferg,None,None,None,None
2030,671789708968640512,NaN,NaN,2015-12-01 20:35:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is space pup. He's very confused. Tries t...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/671789708...,13,10,space,None,None,None,None
1943,673709992831262724,NaN,NaN,2015-12-07 03:45:53 +0000,"<a href=""http://twitter.com/download/iphone"" r...",I know a lot of you are studying for finals. G...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/673709992...,12,10,None,None,None,None,None
161,860563773140209665,NaN,NaN,2017-05-05 18:36:06 +0000,"<a href=""http://twitter.com/download/iphone"" r...",Meet Lorenzo. He's an avid nifty hat wearer an...,NaN,NaN,NaN,https://www.gofundme.com/help-lorenzo-beat-can...,13,10,Lorenzo,None,None,None,None
2093,670780561024270336,NaN,NaN,2015-11-29 01:45:22 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Traviss. He has no ears. Two rare dogs...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/670780561...,7,10,Traviss,None,None,None,None
52,882045870035918850,NaN,NaN,2017-07-04 01:18:17 +0000,"<a href=""http://twitter.com/download/iphone"" r...","This is Koko. Her owner, inspired by Barney, r...",NaN,NaN,NaN,https://twitter.com/dog_rates/status/882045870...,13,10,Koko,None,None,None,None
1725,680070545539371008,NaN,NaN,2015-12-24 17:00:27 +0000,"<a href=""https://about.twitter.com/products/tw...",Say hello to Emmie. She's trapped in an orname...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/680070545...,9,10,Emmie,None,None,None,None


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2356 entries, 0 to 2355
Data columns (total 17 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   tweet_id                    2356 non-null   int64  
 1   in_reply_to_status_id       78 non-null     float64
 2   in_reply_to_user_id         78 non-null     float64
 3   timestamp                   2356 non-null   object 
 4   source                      2356 non-null   object 
 5   text                        2356 non-null   object 
 6   retweeted_status_id         181 non-null    float64
 7   retweeted_status_user_id    181 non-null    float64
 8   retweeted_status_timestamp  181 non-null    object 
 9   expanded_urls               2297 non-null   object 
 10  rating_numerator            2356 non-null   int64  
 11  rating_denominator          2356 non-null   int64  
 12  name                        2356 non-null   object 
 13  doggo                       2356 

In [15]:
df.describe()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,retweeted_status_id,retweeted_status_user_id,rating_numerator,rating_denominator
count,2.356000e+03,7.800000e+01,7.800000e+01,1.810000e+02,1.810000e+02,2356.000000,2356.000000
mean,7.427716e+17,7.455079e+17,2.014171e+16,7.720400e+17,1.241698e+16,13.126486,10.455433
std,6.856705e+16,7.582492e+16,1.252797e+17,6.236928e+16,9.599254e+16,45.876648,6.745237
min,6.660209e+17,6.658147e+17,1.185634e+07,6.661041e+17,7.832140e+05,0.000000,0.000000
25%,6.783989e+17,6.757419e+17,3.086374e+08,7.186315e+17,4.196984e+09,10.000000,10.000000
50%,7.196279e+17,7.038708e+17,4.196984e+09,7.804657e+17,4.196984e+09,11.000000,10.000000
75%,7.993373e+17,8.257804e+17,4.196984e+09,8.203146e+17,4.196984e+09,12.000000,10.000000
max,8.924206e+17,8.862664e+17,8.405479e+17,8.874740e+17,7.874618e+17,1776.000000,170.000000


In [16]:
df[df['text'].str.contains('RT')].shape[0]

192

In [17]:
df[df.tweet_id ==710609963652087808][['tweet_id','expanded_urls','source']]

,tweet_id,expanded_urls,source
1255,710609963652087808,https://vine.co/v/idaTpwH5TgU,"<a href=""http://vine.co"" rel=""nofollow"">Vine -..."


In [18]:
df.name.unique()

array(['Phineas', 'Tilly', 'Archie', 'Darla', 'Franklin', 'None', 'Jax',
       'Zoey', 'Cassie', 'Koda', 'Bruno', 'Ted', 'Stuart', 'Oliver',
       'Jim', 'Zeke', 'Ralphus', 'Canela', 'Gerald', 'Jeffrey', 'such',
       'Maya', 'Mingus', 'Derek', 'Roscoe', 'Waffles', 'Jimbo', 'Maisey',
       'Lilly', 'Earl', 'Lola', 'Kevin', 'Yogi', 'Noah', 'Bella',
       'Grizzwald', 'Rusty', 'Gus', 'Stanley', 'Alfy', 'Koko', 'Rey',
       'Gary', 'a', 'Elliot', 'Louis', 'Jesse', 'Romeo', 'Bailey',
       'Duddles', 'Jack', 'Emmy', 'Steven', 'Beau', 'Snoopy', 'Shadow',
       'Terrance', 'Aja', 'Penny', 'Dante', 'Nelly', 'Ginger', 'Benedict',
       'Venti', 'Goose', 'Nugget', 'Cash', 'Coco', 'Jed', 'Sebastian',
       'Walter', 'Sierra', 'Monkey', 'Harry', 'Kody', 'Lassie', 'Rover',
       'Napolean', 'Dawn', 'Boomer', 'Cody', 'Rumble', 'Clifford',
       'quite', 'Dewey', 'Scout', 'Gizmo', 'Cooper', 'Harold', 'Shikha',
       'Jamesy', 'Lili', 'Sammy', 'Meatball', 'Paisley', 'Albus',
       'Nept

In [19]:
print(df.duplicated(subset=['expanded_urls']).sum())


137


In [20]:
for i in range(len(df[:1])):
    print(df.text.loc[i])

This is Phineas. He's a mystical boy. Only ever appears in the hole of a donut. 13/10 https://t.co/MgUWQ76dJU



#### Qaulity

- Missing values in cloumns `reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, `doggo`, `floofer`, `pupper`, `puppo`, `expanded_urls`
- `text` comlumn contains retweets 
- `name` comlumn cintains Inappropriate names ["a", "the", "None",'O','all','old']
- Incorrect `expanded_urls` for `tweet_id` 812503143955202048 and 710609963652087808
- Incorrect `source`,`expanded_urls` for `tweet_id` 710609963652087808
- imporper datatypes (`in_reply_to_status_id`, `in_reply_to_user_id`, `timestamp`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`)
- Null values are represented as 'None' instead of 'NaN'
- Some tweet ids have invalid `rating_numerator` and `rating_denominator`
- There 137 duplicates

#### Tidiness

- Since we are only interested in looking at orginal tweets, we do not need the following variables(`retweeted_status_id`,`retweeted_status_user_id`,`retweeted_status_timestamp`)
- Some recordes in `source` contain has html tags
- Some tweets' texts in `text` column begain with "RT" followed by a twitter account. 
- `doggo` `floofer` `pupper` `puppo`should be in one Column
- Tweet texts contain URLs at the end

### Assessing `img_pred` table

In [21]:
img_pred.sample(20)

,tweet_id,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog
449,674737130913071104,https://pbs.twimg.com/media/CV0l10AU8AAfg-a.jpg,1,Pomeranian,0.948537,True,schipperke,0.014310,True,Chihuahua,0.008120,True
1302,752917284578922496,https://pbs.twimg.com/media/CnLmRiYXEAAO_8f.jpg,1,German_shepherd,0.609283,True,malinois,0.352460,True,kelpie,0.016105,True
1346,759197388317847553,https://pbs.twimg.com/media/Cok1_sjXgAU3xpp.jpg,1,kuvasz,0.511341,True,golden_retriever,0.076899,True,white_wolf,0.063269,False
2021,880935762899988482,https://pbs.twimg.com/media/DDm2Z5aXUAEDS2u.jpg,1,street_sign,0.251801,False,umbrella,0.115123,False,traffic_light,0.069534,False
1662,811985624773361665,https://pbs.twimg.com/media/C0TAnZIUAAAADKs.jpg,1,Staffordshire_bullterrier,0.610573,True,French_bulldog,0.159935,True,doormat,0.058672,False
419,674038233588723717,https://pbs.twimg.com/media/CVqqMtiVEAEye_L.jpg,1,Eskimo_dog,0.358459,True,Norwegian_elkhound,0.206963,True,malamute,0.148236,True
2039,884925521741709313,https://pbs.twimg.com/media/DEfjEaNXkAAtPlj.jpg,1,Italian_greyhound,0.259916,True,American_Staffordshire_terrier,0.198451,True,Staffordshire_bullterrier,0.127725,True
1503,784826020293709826,https://pbs.twimg.com/media/CuRDF-XWcAIZSer.jpg,1,chow,0.090341,True,binoculars,0.083499,False,Irish_setter,0.077456,True
1776,828376505180889089,https://pbs.twimg.com/media/C378BwxWMAA6CNK.jpg,1,American_Staffordshire_terrier,0.523086,True,Staffordshire_bullterrier,0.186168,True,Chihuahua,0.042089,True
283,671138694582165504,https://pbs.twimg.com/media/CVBdFahXAAAIe5Y.jpg,1,Samoyed,0.587342,True,Great_Pyrenees,0.268952,True,Pekinese,0.090527,True


In [22]:
img_pred.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2075 entries, 0 to 2074
Data columns (total 12 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   tweet_id  2075 non-null   int64  
 1   jpg_url   2075 non-null   object 
 2   img_num   2075 non-null   int64  
 3   p1        2075 non-null   object 
 4   p1_conf   2075 non-null   float64
 5   p1_dog    2075 non-null   bool   
 6   p2        2075 non-null   object 
 7   p2_conf   2075 non-null   float64
 8   p2_dog    2075 non-null   bool   
 9   p3        2075 non-null   object 
 10  p3_conf   2075 non-null   float64
 11  p3_dog    2075 non-null   bool   
dtypes: bool(3), float64(3), int64(2), object(4)
memory usage: 152.1+ KB


In [23]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

img_pred['p1'].value_counts()

golden_retriever                  150
Labrador_retriever                100
Pembroke                           89
Chihuahua                          83
pug                                57
chow                               44
Samoyed                            43
toy_poodle                         39
Pomeranian                         38
cocker_spaniel                     30
malamute                           30
French_bulldog                     26
Chesapeake_Bay_retriever           23
miniature_pinscher                 23
seat_belt                          22
Siberian_husky                     20
German_shepherd                    20
Staffordshire_bullterrier          20
Cardigan                           19
web_site                           19
beagle                             18
Shetland_sheepdog                  18
Eskimo_dog                         18
teddy                              18
Maltese_dog                        18
Lakeland_terrier                   17
Rottweiler  

In [24]:
img_pred.query('tweet_id ==671547767500775424').jpg_url

312    https://pbs.twimg.com/media/CVHRIiqWEAAj98K.jpg
Name: jpg_url, dtype: object

In [25]:
img_pred.isnull().sum()

tweet_id    0
jpg_url     0
img_num     0
p1          0
p1_conf     0
p1_dog      0
p2          0
p2_conf     0
p2_dog      0
p3          0
p3_conf     0
p3_dog      0
dtype: int64

In [26]:
img_pred.jpg_url.duplicated( keep='first').sum()


66

#### Qaulity 
- Underscore sign spearates words in (`p1`, `p2`,`p3`) columns.
- Captalization of Breads of dogs' types in (`p1`, `p2`,`p3`) columns are not consistant. 
- The predictions of dog breed for `tweet_id`s (717790033953034240,675135153782571009,671547767500775424)  are not correct in the three algorithms 
- The dog in image from `tweet_id` 669015743032369152 is not real dog 
- 66 duplicates in `jpg_url` column


### Assessing `tweet_json_df`  table



In [27]:
tweet_json_df.head()

,tweet_id,retweet_count,likes_count
0,892420643555336193,7189,34531
1,892177421306343426,5396,29940
2,891815181378084864,3562,22530
3,891689557279858688,7413,37769
4,891327558926688256,7947,36071


# Cleaning
### Define
- Merge the `df` &`img_pred` & `tweet_json_df` into one master dataframe named `master_df`
- Create a copy of the master dataset

### Code

In [28]:
#  merging the three datasets in one master dataset

master_df = pd.merge(df, img_pred, how = 'left', on = ['tweet_id'] )
master_df = pd.merge(master_df,tweet_json_df, how = 'left', on =['tweet_id'] )
master_df.to_csv('dataset'+'/'+'master_df.csv', encoding='utf-8', index =False)
master_df.head()

,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,likes_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False,7189.0,34531.0
1,892177421306343426,NaN,NaN,2017-08-01 00:17:27 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Tilly. She's just checking pup on you....,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,None,None,None,None,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,5396.0,29940.0
2,891815181378084864,NaN,NaN,2017-07-31 00:18:03 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Archie. He is a rare Norwegian Pouncin...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,None,None,None,None,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1.0,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,3562.0,22530.0
3,891689557279858688,NaN,NaN,2017-07-30 15:58:51 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Darla. She commenced a snooze mid meal...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,None,None,None,None,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1.0,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,7413.0,37769.0
4,891327558926688256,NaN,NaN,2017-07-29 16:00:24 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Franklin. He would like you to stop ca...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,None,None,None,None,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2.0,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,7947.0,36071.0


In [29]:
# Test the dataset file
print(pd.read_csv('dataset/master_df.csv').shape)
pd.read_csv('dataset/master_df.csv').head(1)


(2356, 30)


,tweet_id,in_reply_to_status_id,in_reply_to_user_id,timestamp,source,text,retweeted_status_id,retweeted_status_user_id,retweeted_status_timestamp,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,likes_count
0,892420643555336193,NaN,NaN,2017-08-01 16:23:56 +0000,"<a href=""http://twitter.com/download/iphone"" r...",This is Phineas. He's a mystical boy. Only eve...,NaN,NaN,NaN,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,None,None,None,None,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.07611,False,7189.0,34531.0


In [30]:
# Create a copy dataset from master_df to preserve the original dataset unchanged

master_copy = master_df.copy()
master_copy.shape

(2356, 30)

## Tidiness 


##### Define
- Remove variables(`retweeted_status_id`,`retweeted_status_user_id`,`retweeted_status_timestamp`,`in_reply_to_status_id`,`in_reply_to_user_id`)

#### Code

In [31]:
master_copy.drop(columns=['retweeted_status_id', 'retweeted_status_user_id','retweeted_status_timestamp','in_reply_to_status_id','in_reply_to_user_id'], inplace = True)

#### Test

In [32]:
master_copy.columns

Index(['tweet_id', 'timestamp', 'source', 'text', 'expanded_urls',
       'rating_numerator', 'rating_denominator', 'name', 'doggo', 'floofer',
       'pupper', 'puppo', 'jpg_url', 'img_num', 'p1', 'p1_conf', 'p1_dog',
       'p2', 'p2_conf', 'p2_dog', 'p3', 'p3_conf', 'p3_dog', 'retweet_count',
       'likes_count'],
      dtype='object')

#### Define 
- Remove the HTML tags from `source` column and extract the tweet's source 


#### Code

In [33]:
master_copy.source[4]

'<a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>'

In [34]:
sources = []
for source in master_copy.source:
    sources.append(re.findall(r'>(.*)<',source)[0])
master_copy.source = sources

In [35]:
master_copy.sample(5)

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,doggo,floofer,pupper,puppo,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,likes_count
79,877556246731214848,2017-06-21 15:58:08 +0000,Twitter for iPhone,This is Penny. She's both pupset and fired pup...,https://twitter.com/dog_rates/status/877556246...,12,10,Penny,None,None,None,None,https://pbs.twimg.com/media/DC20wEcW0AAf59m.jpg,1.0,basset,0.995368,True,Welsh_springer_spaniel,0.001936,True,bathtub,0.000468,False,3253.0,20390.0
1958,673580926094458881,2015-12-06 19:13:01 +0000,Twitter for iPhone,When you ask your professor about extra credit...,https://twitter.com/dog_rates/status/673580926...,8,10,None,None,None,None,None,https://pbs.twimg.com/media/CVkKRqOXIAEX83-.jpg,1.0,beagle,0.985062,True,basset,0.006418,True,Walker_hound,0.003533,True,248.0,763.0
455,818307523543449600,2017-01-09 04:04:51 +0000,Twitter for iPhone,RT @dog_rates: Meet Moose. He doesn't want his...,https://twitter.com/dog_rates/status/816450570...,13,10,Moose,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7683.0,0.0
1578,687317306314240000,2016-01-13 16:56:30 +0000,Twitter for iPhone,This is Tyrone. He's a leaf wizard. Self-motiv...,https://twitter.com/dog_rates/status/687317306...,11,10,Tyrone,None,None,None,None,https://pbs.twimg.com/media/CYnXcLEUkAAIQOM.jpg,1.0,Shih-Tzu,0.747208,True,Maltese_dog,0.091025,True,Lhasa,0.035788,True,8604.0,19089.0
180,857062103051644929,2017-04-26 02:41:43 +0000,Twitter for iPhone,RT @AaronChewning: First time wearing my @dog_...,https://twitter.com/AaronChewning/status/85706...,13,10,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,151.0,0.0


##### Test

In [36]:
len(master_copy.source) == len(master_copy)

True

In [37]:
master_copy.source.head()

0    Twitter for iPhone
1    Twitter for iPhone
2    Twitter for iPhone
3    Twitter for iPhone
4    Twitter for iPhone
Name: source, dtype: object

In [38]:
master_copy.source.sample(5)

1520    Twitter for iPhone
2083    Twitter for iPhone
377     Twitter for iPhone
1567    Twitter for iPhone
1803    Twitter for iPhone
Name: source, dtype: object

#### Define 
- Remove Retweets from the dataset 



#### Code

In [39]:
# Count the number of retweet in the dataset

master_copy.text.str.startswith('RT @').sum()

181

In [40]:
# Removie retweets from the dataset

mask = master_copy[master_copy.text.str.startswith('RT @')].index
master_copy.drop(mask ,inplace = True)


#### Test

In [41]:
# Check if there is any retweet in the dataset
master_copy.text.str.startswith('RT @').sum()


0

In [42]:
(len(master_copy)+ 181) == 2356

True


#### Define 

-  Reshape the dataframe by putting those identifier variables into one column `doggo` `floofer` `pupper` `puppo`should be in one Column


#### Code

In [43]:
# putting the columns we want to keep in a list
columns = ['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name','jpg_url',
 'img_num',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'retweet_count',
 'likes_count']



In [44]:
# melt the dog stages columns into one columns 

master_copy = pd.melt(master_copy, id_vars= columns,value_vars= ['doggo', 'floofer', 'pupper', 'puppo'],
            var_name="type", value_name="dog_stage" )

#### Test

In [45]:
master_copy.head()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,likes_count,type,dog_stage
0,892420643555336193,2017-08-01 16:23:56 +0000,Twitter for iPhone,This is Phineas. He's a mystical boy. Only eve...,https://twitter.com/dog_rates/status/892420643...,13,10,Phineas,https://pbs.twimg.com/media/DGKD1-bXoAAIAUK.jpg,1.0,orange,0.097049,False,bagel,0.085851,False,banana,0.076110,False,7189.0,34531.0,doggo,None
1,892177421306343426,2017-08-01 00:17:27 +0000,Twitter for iPhone,This is Tilly. She's just checking pup on you....,https://twitter.com/dog_rates/status/892177421...,13,10,Tilly,https://pbs.twimg.com/media/DGGmoV4XsAAUL6n.jpg,1.0,Chihuahua,0.323581,True,Pekinese,0.090647,True,papillon,0.068957,True,5396.0,29940.0,doggo,None
2,891815181378084864,2017-07-31 00:18:03 +0000,Twitter for iPhone,This is Archie. He is a rare Norwegian Pouncin...,https://twitter.com/dog_rates/status/891815181...,12,10,Archie,https://pbs.twimg.com/media/DGBdLU1WsAANxJ9.jpg,1.0,Chihuahua,0.716012,True,malamute,0.078253,True,kelpie,0.031379,True,3562.0,22530.0,doggo,None
3,891689557279858688,2017-07-30 15:58:51 +0000,Twitter for iPhone,This is Darla. She commenced a snooze mid meal...,https://twitter.com/dog_rates/status/891689557...,13,10,Darla,https://pbs.twimg.com/media/DF_q7IAWsAEuuN8.jpg,1.0,paper_towel,0.170278,False,Labrador_retriever,0.168086,True,spatula,0.040836,False,7413.0,37769.0,doggo,None
4,891327558926688256,2017-07-29 16:00:24 +0000,Twitter for iPhone,This is Franklin. He would like you to stop ca...,https://twitter.com/dog_rates/status/891327558...,12,10,Franklin,https://pbs.twimg.com/media/DF6hr6BUMAAzZgT.jpg,2.0,basset,0.555712,True,English_springer,0.225770,True,German_short-haired_pointer,0.175219,True,7947.0,36071.0,doggo,None


In [46]:
master_copy.dog_stage.value_counts()

None       8344
pupper      234
doggo        87
puppo        25
floofer      10
Name: dog_stage, dtype: int64

In [47]:
master_copy.drop(columns = ['type'], inplace = True)

In [48]:
master_copy = master_copy.sort_values('dog_stage').drop_duplicates('tweet_id', keep = 'last')
master_copy.dog_stage.value_counts()

None       1831
pupper      234
doggo        75
puppo        25
floofer      10
Name: dog_stage, dtype: int64

In [49]:
master_copy.tail()

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,likes_count,dog_stage
7430,738537504001953792,2016-06-03 01:07:16 +0000,Twitter for iPhone,This is Bayley. She fell asleep trying to esca...,https://twitter.com/dog_rates/status/738537504...,11,10,Bayley,https://pbs.twimg.com/media/Cj_P7rSUgAAYQbz.jpg,1.0,chow,0.808737,True,gibbon,0.028942,False,Pembroke,0.026498,True,1436.0,4811.0,puppo
7059,790946055508652032,2016-10-25 16:00:09 +0000,Twitter for iPhone,This is Betty. She's assisting with the dishes...,https://twitter.com/dog_rates/status/790946055...,12,10,Betty,https://pbs.twimg.com/media/CvoBPWRWgAA4het.jpg,1.0,dishwasher,0.700466,False,golden_retriever,0.245773,True,chow,0.039012,True,4504.0,16234.0,puppo
7395,743253157753532416,2016-06-16 01:25:36 +0000,Twitter for iPhone,This is Kilo. He cannot reach the snackum. Nif...,https://twitter.com/dog_rates/status/743253157...,10,10,Kilo,https://pbs.twimg.com/media/ClCQzFUUYAA5vAu.jpg,1.0,malamute,0.442612,True,Siberian_husky,0.368137,True,Eskimo_dog,0.177822,True,1147.0,4024.0,puppo
7276,756275833623502848,2016-07-21 23:53:04 +0000,Twitter for iPhone,When ur older siblings get to play in the deep...,https://twitter.com/dog_rates/status/756275833...,10,10,None,https://pbs.twimg.com/media/Cn7U2xlW8AI9Pqp.jpg,1.0,Airedale,0.602957,True,Irish_terrier,0.086981,True,bloodhound,0.086276,True,1444.0,6098.0,puppo
7298,752519690950500352,2016-07-11 15:07:30 +0000,Twitter for iPhone,Hopefully this puppo on a swing will help get ...,https://twitter.com/dog_rates/status/752519690...,11,10,None,https://pbs.twimg.com/media/CnF8qVDWYAAh0g1.jpg,3.0,swing,0.999984,False,Labrador_retriever,0.000010,True,Eskimo_dog,0.000001,True,3242.0,7026.0,puppo


In [50]:
master_copy.shape

(2175, 22)

#### Define

- Remove URLs from tweet texts

#### Code

In [51]:
string = master_copy.text.iloc[4]

In [52]:

for i in range(len(master_copy.text)):
    h = master_copy.text.iloc[i]
    try:
        indx = h.find(re.search("(?P<url>https?://[^\s]+)", h).group("url"))
        master_copy.text.iloc[i] = h[:indx].strip()
    except:
        master_copy.text.iloc[i] = h.strip()
    


/Users/mozaini/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


#### Test

In [53]:
master_copy.text.sample(5)

2476    Meet Samson. He's absolute fluffy perfection. ...
3148    This is Colby. He's currently regretting all t...
3230              This is Stubert. He just arrived. 10/10
2495    This is Boots. She doesn't know what to do wit...
3223    This is Kanu. He's a Freckled Ticonderoga. Sim...
Name: text, dtype: object

<br>

## Qaulity

- (Done) Missing values in cloumns `reply_to_status_id`, `in_reply_to_user_id`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`, `doggo`, `floofer`, `pupper`, `puppo`, `expanded_urls`
- (Done)`text` comlumn contains retweets 
- (Done) `name` comlumn cintains Inappropriate names ["a", "the", "None",'O','all','old']
- (Done) Incorrect `expanded_urls` for `tweet_id` 812503143955202048 and 710609963652087808
- (Done) Incorrect `source`,`expanded_urls` for `tweet_id` 710609963652087808
- (Done) imporper datatypes (`in_reply_to_status_id`, `in_reply_to_user_id`, `timestamp`, `retweeted_status_id`, `retweeted_status_user_id`, `retweeted_status_timestamp`)
- (Done) Null values are represented as 'None' instead of 'NaN'
- (Done) Some tweet ids have invalid `rating_numerator` and `rating_denominator`
- There 137 duplicates


- The first issue listed under <b>Quality</b> has been delt with when fixing <b>Tidiness</b> issues

- Replace all string None in the dataset with np.nan 
#### Code

In [54]:
# Check nulls before run the code
master_copy.isnull().sum()

tweet_id                0
timestamp               0
source                  0
text                    0
expanded_urls          58
rating_numerator        0
rating_denominator      0
name                    0
jpg_url               181
img_num               181
p1                    181
p1_conf               181
p1_dog                181
p2                    181
p2_conf               181
p2_dog                181
p3                    181
p3_conf               181
p3_dog                181
retweet_count           8
likes_count             8
dog_stage               0
dtype: int64

In [55]:
# Replace all string None in the dataset with np.nan 
master_copy.replace(to_replace="None", value=np.nan, inplace =True)

#### Test

In [56]:
# Check nulls after running the code
master_copy.isnull().sum()

tweet_id                 0
timestamp                0
source                   0
text                     0
expanded_urls           58
rating_numerator         0
rating_denominator       0
name                   680
jpg_url                181
img_num                181
p1                     181
p1_conf                181
p1_dog                 181
p2                     181
p2_conf                181
p2_dog                 181
p3                     181
p3_conf                181
p3_dog                 181
retweet_count            8
likes_count              8
dog_stage             1831
dtype: int64

In [57]:
master_copy.name.isnull().sum()

680

#### Define

- Fix incorrect data types
    - `timestamp` to datetime
    - `img_num` to int
    - `retweet_count` to int
    - `likes_count` to int
    - `p1_dog` to boolen
    - `p2_dog` to boolen
    - `p2_dog` to boolen
    
#### Code

In [58]:
# Check Before converting

master_copy.dtypes

tweet_id                int64
timestamp              object
source                 object
text                   object
expanded_urls          object
rating_numerator        int64
rating_denominator      int64
name                   object
jpg_url                object
img_num               float64
p1                     object
p1_conf               float64
p1_dog                 object
p2                     object
p2_conf               float64
p2_dog                 object
p3                     object
p3_conf               float64
p3_dog                 object
retweet_count         float64
likes_count           float64
dog_stage              object
dtype: object

In [59]:
# Converting 

master_copy.timestamp = pd.to_datetime(master_copy.timestamp)
master_copy['img_num'] = master_copy['img_num'].fillna(0).astype('int')
master_copy['retweet_count'] = master_copy['retweet_count'].fillna(0).astype('int')
master_copy['likes_count'] = master_copy['likes_count'].fillna(0).astype('int')
master_copy.p1_dog = master_copy.p1_dog.astype('bool') 
master_copy.p2_dog = master_copy.p2_dog.astype('bool') 
master_copy.p3_dog = master_copy.p3_dog.astype('bool') 

#### Test

In [60]:
# Check after converting

master_copy.dtypes

tweet_id                            int64
timestamp             datetime64[ns, UTC]
source                             object
text                               object
expanded_urls                      object
rating_numerator                    int64
rating_denominator                  int64
name                               object
jpg_url                            object
img_num                             int64
p1                                 object
p1_conf                           float64
p1_dog                               bool
p2                                 object
p2_conf                           float64
p2_dog                               bool
p3                                 object
p3_conf                           float64
p3_dog                               bool
retweet_count                       int64
likes_count                         int64
dog_stage                          object
dtype: object

#### Define 
- Clean records containing inappropriate names

#### Code

In [61]:
# Check the number of records containing incorrect names
master_copy.query("name == 'a' | name == 'the' | name != name| name == 'O' | name == 'all' | name =='old'")[['name']].count()


name    66
dtype: int64

In [62]:
tweet_texts = list(master_copy.text)
dog_names = list(master_copy.name)

for i in range(len(master_copy)):
    text = tweet_texts[i]
    name = dog_names[i]
    
#     Look for any tweet starts with "This is" and has a caotilized word after it
    if text.startswith("This is ") and re.match(r'[A-Z].*', text.split()[2]):
        if text.split(" ")[2].replace('.',"").strip() != name:
            extracted_name = text.split(" ")[2].strip().replace('.',"").replace(",",'').replace("-"," ")
            dog_names[i] = extracted_name
            master_copy.name.iloc[i] = dog_names[i]
            
#    check for the existing of the word 'named' in tweet's text 
#    compare the word after 'named' with the name in column `name`
    elif 'named' in text and re.match(r'[A-Z].*', text.split()[0]):
            text = text.split()
            named_index = text.index('named')
            extracted_name = text[named_index+1].replace('.','')
            if extracted_name != name:
                master_copy.name.iloc[i]  = extracted_name
#   Look for names that appear after "his/her name"

    elif 'his name ' in text or 'His name ' in text or "Her name " in text or "her name " in text and re.match(r'[A-Z].*', text.split()[0]):
        text = text.split()
        name_index = text.index('name')
        extracted_name = text[name_index+2].replace('.','').replace(",","")
        master_copy.name.iloc[i] = extracted_name
        
#   Look for names after the phrase 'here we have '
#   Compare them with the existing names
    elif 'Here we have ' in text and re.match(r'[A-Z].*', text.split()[0]):
        text = text.split()[3]
        if text != name and re.match(r'[A-Z].*', text.split()[0]):
            master_copy.name.iloc[i] = text
            

    elif "This is a " in text or "This is an " in text and re.match(r'[A-Z].*', text.split()[2]):
        text = text.split()
        if name =='a' or name == 'an' and re.match(r'[A-Z].*', text.split()[0]):
            master_copy.name.iloc[i] = np.nan
    
    elif "This is " in text and re.match(r'[A-Z].*', text.split()[2]):
        text = text.split()
        master_copy.name.iloc[i] = text[2]
#    After doing the above we can savely replace ['the','old','a', all'] with NaN
# Because they the names don't appear in the text
    elif name == 'all' or name == 'the' or name == 'old' or name == 'a':
        master_copy.name.iloc[i] = np.nan
  


#### Test

In [63]:
master_copy.query("name == 'a' | name == 'the' | name != name| name == 'O' | name == 'all' | name =='old'")[['name']].count()

name    0
dtype: int64

#### Define 
- Fix `expanded_urls` for `tweet_id` 812503143955202048 
- Fix `source` for `tweet_id` 710609963652087808
#### Code


In [64]:
master_copy.query('tweet_id == 812503143955202048 | tweet_id ==710609963652087808  ')[['tweet_id','source','expanded_urls']]


,tweet_id,source,expanded_urls
3251,710609963652087808,Vine - Make a Scene,https://vine.co/v/idaTpwH5TgU
2596,812503143955202048,Twitter for iPhone,https://m.facebook.com/story.php?story_fbid=18...


In [65]:
tweet_ids = [812503143955202048, 710609963652087808]
tweet = api.get_status(812503143955202048)
for i in tweet_ids:
    tweet = api.get_status(i)
    if tweet.entities['urls'][0]['expanded_url'] != master_copy.loc[master_copy.tweet_id==i].expanded_urls.to_string(index=False):
        print(tweet.entities['urls'][0]['expanded_url'])
        print(i)
        ind = master_copy.loc[master_copy.tweet_id==i].expanded_urls.index
        master_copy.expanded_urls.loc[ind] = tweet.entities['urls'][0]['expanded_url']


https://twitter.com/i/web/status/812503143955202048
812503143955202048


/Users/mozaini/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [66]:
source = master_copy[master_copy.tweet_id ==710609963652087808 ].source.to_string(index=False).split('-')
source = source[0].strip()
source
master_copy.source[master_copy.tweet_id ==710609963652087808 ]= source

<ipython-input-66-7e9ca0d3aa15>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  master_copy.source[master_copy.tweet_id ==710609963652087808 ]= source


#### Test

In [67]:
master_copy.loc[master_copy.tweet_id==812503143955202048].expanded_urls

2596    https://twitter.com/i/web/status/8125031439552...
Name: expanded_urls, dtype: object

In [68]:
master_copy.query('tweet_id == 812503143955202048 | tweet_id ==710609963652087808  ')[['tweet_id','source','expanded_urls']]


,tweet_id,source,expanded_urls
3251,710609963652087808,Vine,https://vine.co/v/idaTpwH5TgU
2596,812503143955202048,Twitter for iPhone,https://twitter.com/i/web/status/8125031439552...


#### Define 
- Fix invalid `rating_numerator` and `rating_denominator`
#### Code

In [69]:
master_copy.query('rating_numerator <= rating_denominator ').tweet_id.count()

877

In [70]:
# Check how many ratings we can get from tweets
master_copy.text.apply(lambda x: re.findall(r'(\d+/\d+)', text)).count()


2175

In [71]:

for i in range(len(master_copy)):
    text = master_copy.text.iloc[i]
    rate = re.findall(r'(\d+/\d+)', text)
    old_rate = str(master_copy.rating_numerator.iloc[i])+ '/'+ str(master_copy.rating_denominator.iloc[i])
# It looks that some tweets have more than one rating and rating caputured in the
# the dataset is the first rating
# The solution to this can be taking the average of the numerator of the extracted ratings
        
    if len(rate) > 1:
      
        if len(rate) == 2:
            numerator = (int(rate[0].split('/')[0])+int(rate[1].split('/')[0]))/2
            master_copy.rating_numerator.iloc[i] = numerator
#             print(str(numerator)+'/'+str(master_copy.rating_denominator.iloc[i]),' ', rating)
        elif len(rate) ==3:
            numerator = (int(rate[0].split('/')[0])+int(rate[1].split('/')[0])+ int(rate[2].split('/')[0]))/3
            master_copy.rating_numerator.iloc[i] = round(numerator,1)
        else:
            master_copy.rating_numerator.iloc[i] = np.nan
            master_copy.rating_denominator.iloc[i] = np.nan

/Users/mozaini/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


- After checking the records containing a numerator greater than the denominator, most of those recorders matched the actual rating mentioned in the tweets. The others that did not match the rating in the tweets' texts were due to the existence of more than one dog's rating. 

#### Define
- Remove Underscores from (`p1`, `p2`,`p3`)
#### Code

In [72]:
master_copy[['p1','p2','p3']] = master_copy[['p1','p2','p3']].apply(lambda x: x.str.replace("_"," "))


#### Test


In [73]:
master_copy[['p1','p2','p3']].head()

,p1,p2,p3
2095,goose,drake,hen
2094,Labrador retriever,French bulldog,Staffordshire bullterrier
2093,Tibetan terrier,Irish terrier,Yorkshire terrier
2092,toy poodle,pug,Lakeland terrier
2091,borzoi,ice bear,weasel


In [74]:
master_copy[['p1','p2','p3']].tail()

,p1,p2,p3
7430,chow,gibbon,Pembroke
7059,dishwasher,golden retriever,chow
7395,malamute,Siberian husky,Eskimo dog
7276,Airedale,Irish terrier,bloodhound
7298,swing,Labrador retriever,Eskimo dog


#### Define 
- Captalize dogs' breeds in in columns (`p1`, `p2`,`p3`) 
#### Code

In [75]:
master_copy[['p1','p2','p3']] = master_copy[['p1','p2','p3']].apply(lambda x: x.str.capitalize())



#### Test

In [76]:
master_copy[['p1','p2','p3']].head()

,p1,p2,p3
2095,Goose,Drake,Hen
2094,Labrador retriever,French bulldog,Staffordshire bullterrier
2093,Tibetan terrier,Irish terrier,Yorkshire terrier
2092,Toy poodle,Pug,Lakeland terrier
2091,Borzoi,Ice bear,Weasel


In [77]:
master_copy[['p1','p2','p3']].tail()

,p1,p2,p3
7430,Chow,Gibbon,Pembroke
7059,Dishwasher,Golden retriever,Chow
7395,Malamute,Siberian husky,Eskimo dog
7276,Airedale,Irish terrier,Bloodhound
7298,Swing,Labrador retriever,Eskimo dog


#### Define
- Remove duplicates form  `jpg_url` column
#### code

In [78]:
master_copy[master_copy.jpg_url.duplicated()]

,tweet_id,timestamp,source,text,expanded_urls,rating_numerator,rating_denominator,name,jpg_url,img_num,p1,p1_conf,p1_dog,p2,p2_conf,p2_dog,p3,p3_conf,p3_dog,retweet_count,likes_count,dog_stage
81,875097192612077568,2017-06-14 21:06:43+00:00,Twitter for iPhone,You'll get your package when that precious man...,https://twitter.com/drboondoc/status/874413398...,13.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,5249,24760,NaN
76,876537666061221889,2017-06-18 20:30:39+00:00,Twitter for iPhone,I can say with the pupmost confidence that the...,https://twitter.com/mpstowerham/status/8761629...,14.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,4070,21451,NaN
29,886267009285017600,2017-07-15 16:51:35+00:00,Twitter for iPhone,@NonWhiteHat @MayhewMayhem omg hello tanner yo...,NaN,12.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,3,104,NaN
68,878604707211726852,2017-06-24 13:24:20+00:00,Twitter for iPhone,Martha is stunning how h*ckin dare you. 13/10,https://twitter.com/bbcworld/status/8785998685...,13.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,6063,26987,NaN
33,885518971528720385,2017-07-13 15:19:09+00:00,Twitter for iPhone,I have a new hero and his name is Howard. 14/10,https://twitter.com/4bonds2carbon/status/88551...,14.0,10,Howard,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,3190,18389,NaN
61,879674319642796034,2017-06-27 12:14:36+00:00,Twitter for iPhone,@RealKentMurphy 14/10 confirmed,NaN,14.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,9,284,NaN
52,881633300179243008,2017-07-02 21:58:53+00:00,Twitter for iPhone,@roushfenway These are good dogs but 17/10 is ...,NaN,15.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,8,113,NaN
39,884247878851493888,2017-07-10 03:08:17+00:00,Twitter for iPhone,OMG HE DIDN'T MEAN TO HE WAS JUST TRYING A LIT...,https://twitter.com/kaijohnson_19/status/88396...,13.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,17343,64789,NaN
3511,690989312272396288,2016-01-23 20:07:44+00:00,Vine - Make a Scene,We've got a doggy down. Requesting backup. 12/...,https://vine.co/v/iOZKZEU2nHq,12.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,2692,5600,NaN
3501,691793053716221953,2016-01-26 01:21:31+00:00,Vine - Make a Scene,We usually don't rate penguins but this one is...,https://vine.co/v/OTTVAKw6YlW,10.0,10,NaN,NaN,0,NaN,NaN,True,NaN,NaN,True,NaN,NaN,True,3958,7692,NaN


In [79]:
tweet = api.get_status(704134088924532736)
tweet.entities

{'hashtags': [],
 'symbols': [],
 'user_mentions': [],
 'urls': [{'url': 'https://t.co/h5aI0Tim4j',
   'expanded_url': 'https://vine.co/v/igW2OEwu9vg',
   'display_url': 'vine.co/v/igW2OEwu9vg',
   'indices': [64, 87]}]}

#### Result
- After inspecting the column `jpg_url,` I have concluded that the duplicated values are resulted from
    - The absence of media URLs in the source 
    - A Copyright issue with the original tweets
    - Invalid tweet's URLs
    - Inline images in the original tweets
    - Deleted tweets
- As a result, `'jpg_url` column has been filled with null due to the above issues
- However, since those records still contain some information that I can use in my Analysis, such as ratings, I decided to keep them 

#### Define
- Remove records with incorrect predictions and invalid dogs picutres 
#### Code 

In [80]:

indeces = master_copy.query('tweet_id == 717790033953034240 | tweet_id == 675135153782571009 | tweet_id==671547767500775424 | tweet_id ==669015743032369152').index

In [81]:
master_copy.drop(indeces, inplace = True ) 

#### Test

In [83]:
master_copy.shape

(2171, 22)

#### Define 
- Drop unnecessary columns [`jpg_url`,`img_num`]
#### Code

In [84]:
master_copy.drop(columns=['jpg_url','img_num'], inplace = True)

#### Test

In [88]:
list(master_copy)


['tweet_id',
 'timestamp',
 'source',
 'text',
 'expanded_urls',
 'rating_numerator',
 'rating_denominator',
 'name',
 'p1',
 'p1_conf',
 'p1_dog',
 'p2',
 'p2_conf',
 'p2_dog',
 'p3',
 'p3_conf',
 'p3_dog',
 'retweet_count',
 'likes_count',
 'dog_stage']

# Store Data

In [ ]:
master_copy.to_csv('dataset/twitter_archive_master.csv', index = False)